In [ ]:
import json
import pandas as pd
import numpy as np

import my_pickle as mp

In [ ]:
filename = "/Users/gandalf/Documents/data/data_users.json"
# with open(filename) as fh:
#     json_data = fh.read()
#     data = json.loads(json_data)

In [ ]:
df = pd.read_json(filename)

In [ ]:
backup = df.copy()

In [ ]:
df = backup.copy()

In [ ]:
df.head(2)

In [ ]:
df.columns

    '_acl'                drop     UNKNOWN, ex: {'*': {'r': True}, '25mzfm6r5U': {'r': True, 'w': True}}
    '_auth_data_facebook' drop     FACEBOOK AUTHORIZATION, not helpful (maybe as a yes/no?)
    '_created_at'         to_date  DATE CREATED, format {'$date': '2016-11-15T22:54:02.653Z'}
    '_hashed_password'    PASSWORD, not useful
    '_id'                 string, IMPORTANT!
    '_p_room'             UNKNOWN, string, ex: 'Room$dANkaNpTN9' post a room, someone with a room available
    '_rperm'              UNKNOWN, ['*', '0PZ0jSiV11'] ignore
    '_updated_at'         DATE UPDATED, eg: {'$date': '2017-03-13T19:38:46.405Z'}
    '_wperm'              UNKNOWN, eg: ['0PZ0jSiV11'] ignore
    'about'               ABOUT ME SECTION, string
    'activeAt'            MOST RECENT ACTIVE format: {'$date': '2017-12-04T03:01:37.052Z'}
    'amenities'           WHICH AMENITIES, eg: [Laundry, A/C, Dishwasher, Private Bath]
    'available'           WHEN AVAILABLe, format: {'$date': '2017-01-31T11:40:52.000Z'}
    'birthday'            BIRTHDAY, eg: {'$date': '1992-01-01T00:00:00.000Z'}
    'blocked'             EVER BLOCKED eg: [{'__type': 'Pointer', 'className': '_User', 'objectId': 'nHcaVTlJUg'}]
    'college'             COLLEGE, string
    'email'               EMAIL, string
    'emailVerified'       ???, just NaN and 0 ignore
    'facebookId'          18-long num, or NaN
    'firstName'           FIRST NAME (string)
    'foundRoommate'       NaN or 1 (ignore)
    'gender'              GENDER, male, female, nan
    'groupChat'           ??? NaN or 1
    'hobbies'             HOBBIES eg: [Foodie, Movies, Socializing]
    'hometown'            HOMETOWN, mostly NAN, has city, state, country
                          'hometownCity', 'hometownCountry','hometownCounty', 'hometownState'
    'inRelationship', 
    'isClean',
    'isNight', 
    'isStudent', 
    'lastName', 
    'likes'               CHECK BACK facebook likes,eg a band or place
    'linkedinId'
    'location'            LATITUDE AND LONGITUDE
    'maxCost', 'minCost', 
    'neighborhoods'       eg: [{'__type': 'Pointer', 'className': 'Neighborhood', 'objectId': 'rQgVwGMItl'}]
    'numRoommates' 
    'onboarded'           SUPA IMPORT!!!
    'petsOk', 
    'picture'             string, eg: '6c5316ca545062ad708899e89adb917a_pic.jpeg'
    'positions'           some sort of list, picture positions, for posting a room
    'recommended'         ignore, internal use, for caching users
    'smokingOk'
    'term',
    'type',               house type ( 
    'username', 
    'work']
    

In [ ]:
col_to_drop = ['_acl','_auth_data_facebook','_hashed_password','_rperm','_wperm','blocked','email','emailVerified','firstName',
                  'foundRoommate','groupChat','hometown','hometownCounty','likes','lastName','positions','recommended','username']
df = df.drop(col_to_drop, axis = 1)

df = df.fillna({'about':''})

df = df.rename(index=str, columns={"_created_at": "created",
                                   "_updated_at": "updated",
                                   "_p_room"    : "has_room",
                                   "_id"        : "uid"})
    
df = df.set_index('uid')   
df['len_about'] = df.about.apply(lambda x: len(x))
df['has_about'] = df.len_about > 0
df.has_room = df.has_room.apply(lambda x: isinstance(x,str))
df.facebookId = df.facebookId.apply(lambda x: isinstance(x,str))
df.linkedinId = df.linkedinId.apply(lambda x: isinstance(x,str))
df.picture = df.picture.apply(lambda x: isinstance(x,str))


def my_to_datetime(x):
    if isinstance(x, dict):
        try:
            return pd.to_datetime(x['$date'])
        except:
            return None
    else:
        return None
df.created = df.created.apply(lambda x: my_to_datetime(x))
df.updated = df.updated.apply(lambda x: my_to_datetime(x))
df.activeAt = df.activeAt.apply(lambda x: my_to_datetime(x))
df.available = df.available.apply(lambda x: my_to_datetime(x))
df.birthday = df.birthday.apply(lambda x: my_to_datetime(x))
df['age'] = 2018-df['birthday'].apply(lambda x: x.year)

df = df[['created', 'updated', 'activeAt', 'available', # dates
         'about', 'has_about', 'len_about',            # about
         'birthday', 'age', 'gender', 'location', 'work',      # demographic
         'hometownCity', 'hometownCountry', 'hometownState',     # more demographic
         'college', 'facebookId','linkedinId', 'picture',        # engagement
         'maxCost', 'minCost', 'neighborhoods', 'numRoommates', 'term', 'type', 'has_room',  # room basics
         'amenities', 'hobbies',                                              # room not boolean
         'inRelationship', 'isClean', 'isNight', 'isStudent', 'petsOk', 'smokingOk',   # room boolean
         'onboarded',]]

In [ ]:
df.head()

In [ ]:
df.likes[df.likes.apply(lambda x: isinstance(x, list))][7]

In [ ]:
print(len(df))

In [ ]:
type(df.birthday[0]['$date'])

In [ ]:
def get_user_from_json(df):
    '''
    original columns: 
    '_acl'                drop     UNKNOWN, ex: {'*': {'r': True}, '25mzfm6r5U': {'r': True, 'w': True}}
    '_auth_data_facebook' drop     FACEBOOK AUTHORIZATION, not helpful (maybe as a yes/no?)
    '_created_at'         to_date  DATE CREATED, format {'$date': '2016-11-15T22:54:02.653Z'}
    '_hashed_password'    PASSWORD, not useful
    '_id'                 string, IMPORTANT!
    '_p_room'             UNKNOWN, string, ex: 'Room$dANkaNpTN9' post a room, someone with a room available
    '_rperm'              UNKNOWN, ['*', '0PZ0jSiV11'] ignore
    '_updated_at'         DATE UPDATED, eg: {'$date': '2017-03-13T19:38:46.405Z'}
    '_wperm'              UNKNOWN, eg: ['0PZ0jSiV11'] ignore
    'about'               ABOUT ME SECTION, string
    'activeAt'            MOST RECENT ACTIVE format: {'$date': '2017-12-04T03:01:37.052Z'}
    'amenities'           WHICH AMENITIES, eg: [Laundry, A/C, Dishwasher, Private Bath]
    'available'           WHEN AVAILABLe, format: {'$date': '2017-01-31T11:40:52.000Z'}
    'birthday'            BIRTHDAY, eg: {'$date': '1992-01-01T00:00:00.000Z'}
    'blocked'             EVER BLOCKED eg: [{'__type': 'Pointer', 'className': '_User', 'objectId': 'nHcaVTlJUg'}]
    'college'             COLLEGE, string
    'email'               EMAIL, string
    'emailVerified'       ???, just NaN and 0 ignore
    'facebookId'          18-long num, or NaN
    'firstName'           FIRST NAME (string)
    'foundRoommate'       NaN or 1 (ignore)
    'gender'              GENDER, male, female, nan
    'groupChat'           ??? NaN or 1
    'hobbies'             HOBBIES eg: [Foodie, Movies, Socializing]
    'hometown'            HOMETOWN, mostly NAN, has city, state, country
                          'hometownCity', 'hometownCountry','hometownCounty', 'hometownState'
    'inRelationship', 
    'isClean',
    'isNight', 
    'isStudent', 
    'lastName', 
    'likes'               CHECK BACK facebook likes,eg a band or place
    'linkedinId'
    'location'            LATITUDE AND LONGITUDE
    'maxCost', 'minCost', 
    'neighborhoods'       eg: [{'__type': 'Pointer', 'className': 'Neighborhood', 'objectId': 'rQgVwGMItl'}]
    'numRoommates' 
    'onboarded'           SUPA IMPORT!!!
    'petsOk', 
    'picture'             string, eg: '6c5316ca545062ad708899e89adb917a_pic.jpeg'
    'positions'           some sort of list, picture positions, for posting a room
    'recommended'         ignore, internal use, for caching users
    'smokingOk'
    'term',
    'type',               house type ( 
    'username', 
    'work']
    
    '''
    
    df._created_at = df._created_at.apply(lambda x: pd.to_datetime(x['$date']))
    df._p_room = df._p_room.apply(lambda x: isinstance(x,str))
    df._updated_at = df._updated_at.apply(lambda x: pd.to_datetime(x['$date']))
    df.activeAt = df.activeAt.apply(lambda x: pd.to_datetime(x['$date']) if isinstance(x,str) else 0)
    df.available = df.available.apply(lambda x: pd.to_datetime(x['$date']) if isinstance(x,str) else 0)
    df.birthday = df.birthday.apply(lambda x: x['$date'] if isinstance(x,str) else 0)

#     df.birthday = df.birthday.apply(lambda x: pd.to_datetime(x['$date']) if isinstance(x,str) else 0)
    df.email = df.email.apply(lambda x: True if isinstance(x,str) else False)
    df.facebookId = df.facebookId.apply(lambda x: True if isinstance(x,int) else False)
    df.likes = df.likes.apply(lambda x: True if isinstance(x,int) else False)
    df.linkedinId = df.linkedinId.apply(lambda x: True if isinstance(x,int) else False)
    df.picture = df.picture.apply(lambda x: True if isinstance(x,str) else False)
    df['work_tf'] = df.work.apply(lambda x: True if isinstance(x,str) else False)
    
    
    col_to_drop = ['_acl','_auth_data_facebook','_hashed_password','_rperm','_wperm','blocked','emailVerified','firstName',
                  'foundRoommate','groupChat','hometown','hometownCounty','lastName','positions','recommended','username']
    df = df.fillna({'about':''})

    df = df.drop(col_to_drop, axis = 1)
    
    df = df.rename(index=str, columns={"_created_at": "created",
                                       "_updated_at": "updated",
                                       "_p_room"    : "has_room",
                                       "_id"        : "uid"})
    
    df['len_about'] = df.about.apply(lambda x: len(x))
    df['has_about'] = df.len_about > 0
#     df['age'] = 2018-df['birthday'].apply(lambda x: x.year)
 
    df = df[df.onboarded==1]
    
    df = df.set_index('uid')
    
    return df

temp_df = df.copy()
temp_df = get_user_from_json(temp_df)

In [ ]:
temp_df.iloc[:,:].head()

In [ ]:
temp_df.birthday

In [ ]:
print(len(temp_df))

firebase - conversations
mongodb for others